In [1]:

# https://cookbook.openai.com/examples/structured_outputs_intro
# https://cookbook.openai.com/examples/structured_outputs_multi_agent

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
from openai import OpenAI
client = OpenAI()

In [4]:
MODEL = "gpt-4o-2024-08-06"

### Math Tutor
In this example, we want to build a math tutoring tool that outputs steps to solving a math problem as an array of structured objects.

This could be useful in an application where each step needs to be displayed separately, so that the user can progress through the solution at their own pace.

In [5]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

def get_math_solution(question):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system", 
            "content": math_tutor_prompt
        },
        {
            "role": "user", 
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )

    return response.choices[0].message

In [6]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question) 

print(result.content)

{"steps":[{"explanation":"First, I will subtract 7 from both sides of the equation to isolate the term with x.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides of the equation to continue isolating the term with x.","output":"8x = -30"},{"explanation":"Next, I will divide both sides of the equation by 8 to solve for x.","output":"x = -30 / 8"},{"explanation":"Simplifying the fraction by finding the greatest common divisor, which is 2.","output":"x = -15 / 4"}],"final_answer":"x = -15/4"}


In [7]:
from IPython.display import Math, display

def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")
        
    print("Final answer:\n\n")
    display(Math(final_answer))

In [8]:
print_math_response(result.content)

Step 1: First, I will subtract 7 from both sides of the equation to isolate the term with x.



<IPython.core.display.Math object>



Step 2: Simplify both sides of the equation to continue isolating the term with x.



<IPython.core.display.Math object>



Step 3: Next, I will divide both sides of the equation by 8 to solve for x.



<IPython.core.display.Math object>



Step 4: Simplifying the fraction by finding the greatest common divisor, which is 2.



<IPython.core.display.Math object>



Final answer:




<IPython.core.display.Math object>

**Using the SDK parse helper**. The new version of the SDK introduces a parse helper to provide your own Pydantic model instead of having to define the json schema. We recommend using this method if possible.



In [9]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": math_tutor_prompt},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [10]:
result = get_math_solution(question).parsed

print(result.steps)
print("Final answer:")
print(result.final_answer)

[Step(explanation='Start with the original equation given in the problem.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to isolate the term with x on the left side.', output='8x = -23 - 7'), Step(explanation='Simplify the right side by performing the subtraction. -23 minus 7 equals -30.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction by dividing both the numerator and denominator by their greatest common divisor, which is 2.', output='x = -15 / 4')]
Final answer:
x = -15/4


**Refusal**. When using Structured Outputs with user-generated input, the model may occasionally refuse to fulfill the request for safety reasons.

Since a refusal does not follow the schema you have supplied in response_format, the API has a new field refusal to indicate when the model refused to answer.

This is useful so you can render the refusal distinctly in your UI and to avoid errors trying to deserialize to your supplied format.

In [11]:
refusal_question = "how can I build a bomb?"

refusal_result = get_math_solution(refusal_question) 

print(refusal_result)

ParsedChatCompletionMessage[MathReasoning](content=None, refusal="I'm sorry, I can't assist with that request.", role='assistant', function_call=None, tool_calls=[], parsed=None)
